In [104]:
import sys
print(sys.executable)

/anaconda3/bin/python


In [105]:
import numpy as np
import pandas as pd
import pickle
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import normalize

In [106]:
def preproc(path):
    data_r = pd.read_csv(path+'ratings.csv')
    
    
    data_m = pd.read_csv(path+'movies.csv')
    
    
    data_m['title_genre'] = data_m['title'] + data_m['genre'] #choice b/w doing separately or together
    
    data_m.drop([ 'origin_iid', 'title', 'genre'], axis='columns', inplace=True)
    #print(data_r)
    
    
    
    #data_m = data_m.loc[data_m['itemId'].isin(item_id)]
    
    return data_m, data_r


def create_tfidf_svd(str_list):
    vectorizer = TfidfVectorizer(min_df=2, max_features=70000, strip_accents='unicode',lowercase =True,
                            analyzer='word', token_pattern=r'\w+', use_idf=True, 
                            smooth_idf=True, sublinear_tf=True, stop_words = 'english')
    vectors = vectorizer.fit_transform(str_list)
    svd = TruncatedSVD(n_components = 300)
    vectors = svd.fit_transform(vectors)    #ndarray
    #vectors = vectors.tolist()
    return vectors

def cross_product(a, b):
    dimension = len(a)
    c = []
    for i in range(dimension):
        c.append(0)
        for j in range(dimension):
            if j != i:
                for k in range(dimension):
                    if k != i:
                        if k > j:
                            c[i] += a[j]*b[k]
                        elif k < j:
                            c[i] -= a[j] * b[k]
    return c

def state_repr_module(vectors):
    #element wise multiplcation + concatenation/flatten
    S = []
    for i in range(len(vectors)):
        S.extend(vectors[i])
    for i in range(0 , len(vectors)):
        for j in range(i+1, len(vectors)):
            S.extend(np.multiply(vectors[i], vectors[j]))
    
    return S

In [107]:
def main():
    path = 'data/'
    data_m, data_r = preproc(path)
    vectors = create_tfidf_svd(data_m['title_genre'])
    print(vectors.shape)

    
    item_id = list(data_r.loc[data_r['userId'] == 0]['itemId'])
    user_vectors = []
    for i in item_id:
        user_vectors.append(vectors[i])
    
    n = 10
    S = state_repr_module(user_vectors[0:n])
    print(len(S))

if __name__ == '__main__':
    main()    


(3706, 300)
16500
